In [ ]:
''' Copyright 2019 Xilinx Inc.

Licensed under the Apache License, Version 2.0 (the "License"); you may 
not use this file except in compliance with the License. You may obtain
a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
'''

In [ ]:
# The model for the skin cancer classifier.
# Alimul, Xilinx Inc, Dec, 2019
# trained with HAM10000 dataset. 7 types of skin lesions. 

# - Actinic Keratoses
# - Basal Cell Carcinoma
# - Benign Keratosis
# - Dermatofibroma
# - Malignant Melanoma
# - Melanocytic Nevi
# - Vascular Lesions


# Import the libraries
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras.layers.core import Dense, Dropout
from keras.layers import Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from freeze_session import freeze_session


In [ ]:
print(tf.__version__)
# Check if GPU is available
K.tensorflow_backend._get_available_gpus()

In [ ]:
#  The paths for the training and validation images
train_path = 'data/train_val/train_dir'
valid_path = 'data/train_val/val_dir'

# Declare a few hyperparameters 
num_train_samples = 42154
num_val_samples = 3002
train_batch_size = 10
val_batch_size = 10
image_size = 224

# Steps are needed in an iteration
train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

In [ ]:
# generators
train_batches = ImageDataGenerator(
    preprocessing_function= \
        keras.applications.inception_v3.preprocess_input).flow_from_directory(
    train_path,
    target_size=(image_size, image_size),
    batch_size=train_batch_size,
    shuffle=True)

valid_batches = ImageDataGenerator(
    preprocessing_function= \
        keras.applications.inception_v3.preprocess_input).flow_from_directory(
    valid_path,
    target_size=(image_size, image_size),
    batch_size=val_batch_size)

test_batches = ImageDataGenerator(
    preprocessing_function= \
        keras.applications.inception_v3.preprocess_input).flow_from_directory(
    valid_path,
    target_size=(image_size, image_size),
    batch_size=val_batch_size,
    shuffle=False)

In [ ]:
tcl =train_batches.classes
print(tcl.shape)
vcl = valid_batches.classes
print(vcl)
tscl = test_batches.classes
print(tscl)
print(test_batches.class_indices.keys())
print(valid_batches.class_indices.values())
print(test_batches.labels)

In [ ]:
# Create a inception_v3 model alonge with weights
iv3_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# See a summary of the layers in the model
iv3_model.summary()

In [ ]:
# Taking the output of the inception_v3 just before last layer
x = iv3_model.output
# flattening the outputs of the last conv layer
flatten = Flatten()(x)
# adding two fully connected layers. Meeting DPU requirement, keeping output/input ratio at @ 1/6
dense1 = Dense(2048, activation= 'relu')(flatten)
dense3= Dense(128, activation= 'relu')(dense1)
# adding the prediction layer with 'softmax'
predictions = Dense(7, activation='softmax')(dense3)

# Create a new model with the new outputs
model = Model(inputs=iv3_model.input, outputs=predictions)

# See a summary of the new layers in the model
model.summary()

In [ ]:
# Freeze the weights of the layers that aren't training
for layer in model.layers[:-3]:
    layer.trainable = False
#for layer in model.layers:
    #print(layer.name, layer.trainable)

In [ ]:
# Train the model
# Define Top2 and Top3 Accuracy
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy

# Compile the model
#model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=[categorical_accuracy, top_2_accuracy, top_3_accuracy])
model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=[categorical_accuracy])

# Add weights to make the model more sensitive to melanoma
class_weights={
    0: 1.0,  # akiec
    1: 1.0,  # bcc
    2: 1.0,  # bkl
    3: 1.0,  # df
    4: 3.0,  # mel
    5: 1.0,  # nv
    6: 1.0,  # vasc
}

# Declare the filepath for the saved model
filepath = "model/model_incv3_3_12.h5"

# Declare a checkpoint to save the best version of the model
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1,
                             save_best_only=True, mode='max')

# Reduce the learning rate as the learning stagnates
reduce_lr = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.2, patience=2,
                              verbose=1, mode='max', min_lr=0.000001)
callbacks_list = [checkpoint, reduce_lr]

# Fit the model
history = model.fit_generator(train_batches,
                              steps_per_epoch=train_steps,
                              class_weight=class_weights,
                              validation_data=valid_batches,
                              validation_steps=val_steps,
                              epochs=30,
                              verbose=1,
                              callbacks=callbacks_list)

In [ ]:
# Retrain the trained model

# Define Top2 and Top3 Accuracy
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy

# Add weights to make the model more sensitive to melanoma
class_weights={
    0: 1.0,  # akiec
    1: 1.0,  # bcc
    2: 1.0,  # bkl
    3: 1.0,  # df
    4: 2.0,  # mel
    5: 1.0,  # nv
    6: 1.0,  # vasc
}

# Declare the filepath for the saved model
filepath = "model/model_incv3_2_6.h5"

model=keras.models.load_model(filepath, custom_objects=dependencies)
model.summary()

# Declare a checkpoint to save the best version of the model
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1,
                             save_best_only=True, mode='max')

# Reduce the learning rate as the learning stagnates
reduce_lr = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=2,
                              verbose=1, mode='max', min_lr=0.0000001)

callbacks_list = [checkpoint, reduce_lr]

# Fit the model
history = model.fit_generator(train_batches,
                              steps_per_epoch=train_steps,
                              class_weight=class_weights,
                              validation_data=valid_batches,
                              validation_steps=val_steps,
                              epochs=20,
                              verbose=1,
                              callbacks=callbacks_list)

In [ ]:
# Evaluation of the best epoch
#model.load_weights('model/model_incv3_3_12.h5')
model=keras.models.load_model('model/model_incv3_3_13.h5')

val_loss, val_cat_acc = \
model.evaluate_generator(test_batches, steps=val_steps)

print('val_loss:', val_loss)
print('val_cat_acc:', val_cat_acc)


In [ ]:
score = model.evaluate_generator(test_batches, steps=val_steps)
print(model.metrics_names, score)

In [ ]:
print('Output node:', [out.op.name for out in model.outputs])
print('Input node:',[inp.op.name for inp in model.inputs])

In [ ]:
# save the model as .pb file

pb_file = "model/model_incv3_3_13.pb"
frozen_graph = freeze_session(K.get_session(),output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, ".", pb_file , as_text=False)